# Reading & Writing to the Database

In [32]:
import DatabaseFunctions
import importlib
import time
importlib.reload(DatabaseFunctions)

# The "Super" class upon which the other application are built
# Won't need to directly call this as its looped into whichever application you're using
DBF = DatabaseFunctions.DatabaseFunctions()
# print('Data-Years by Site')
# DBF.years_by_site

# # Given an ini file with a request, can create database traces from input files (e.g., .dat, .csv, etc.)
# WD = DatabaseFunctions.MakeTraces(ini='ini_files/WriteTraces.ini')

# # Given an ini file with a request, can create database traces from google sheets
# # Good for manual data collected in the field (e.g. manaul WTD data)
# # Assumes local (daylight/standard) time, will auto convert to local standard time
# WG = DatabaseFunctions.GSheetDump(ini='ini_files/WriteTraces_Gsheets.ini')

# # Given an ini file with a request, can create .csv files from database traces
# # By default will process request for each site/year
# # Passing Sites =['BB','BB2'] or Years = [2022,2023] etc. can be used to restrict request
# RD = DatabaseFunctions.MakeCSV(Sites = ['BBS'])



Initialized using db_root:  C:/Database/


In [33]:
DBF.ini

{'Database': {'Paths': {'SecondStage': 'Clean/SecondStage',
   'ThirdStage': 'Clean/ThirdStage'},
  'datenum': {'filename': 'clean_tv',
   'dtype': 'float64',
   'datenum_base': 719529,
   'datenum_base_unit': 'D'},
  'traces': {'dtype': 'float32'}}}

In [52]:
import json
import configparser

conf = configparser.ConfigParser()
conf.read('ini_files/Write_CSV_Files.ini')

Biom = dict(conf['FFP_Inputs'])

Trace_Deffs = {'Traces':{}}
# for in_name,out_name,units in zip(Biom['traces'].split(','),Biom['rename'].split(','),Biom['units'].split(',')):
for in_name,units in zip(Biom['traces'].split(','),Biom['units'].split(',')):
    Trace_Deffs['Traces'][in_name] = {'Units':units,'output_name':in_name}
Trace_Deffs['timestamp'] = {'timestamp_fmt':Biom['timestamp_fmt'],'timestamp_units':Biom['timestamp_units'],'output_name':Biom['timestamp']}
for i in [ 'units_in_header','output_path','by_year','stage']:
    Trace_Deffs[i] = Biom[i]
Trace_Deffs['na_value'] = 'np.nan'
Trace_Deffs['filename'] = 'np.nan'
# Biom
with open('ini_files/Write_FFP_Input_CSV.json', 'w') as f:
    json.dump(Trace_Deffs, f,indent=2)

# with open('ini_files/Write_Biomet_CSV.json') as jsonfile:
#     # `json.loads` parses a string in json format
#     Trace_Deffs = json.load(jsonfile)
Biom

{'output_path': 'HIGHFREQ/footprint/',
 'by_year': 'False',
 'stage': 'Clean/SecondStage/',
 'traces': 'L,USTAR,wind_speed,wind_dir,V_SIGMA,canopy_height,hpbl',
 'units': 'm,m s^-1,m s^-1,deg,m s^-1,m,m',
 'timestamp': 'TIMESTAMP',
 'timestamp_fmt': '%Y-%m-%d %H%M',
 'timestamp_units': 'yyyy-mm-dd HHMM',
 'units_in_header': 'False',
 'rename': ''}

# Downloading NARR 

* Can expand to ECCC Data and various other products if needed

In [13]:
import ExtractNARR
import importlib
import time
importlib.reload(ExtractNARR)
T1 = time.time()
EX = ExtractNARR.PointSample([2021,2022,2023],Limit_to_Sites=['DM','RM','NM','NS','NOB','NFB'])
print(time.time()-T1)

Estimating hpbl for 2021 at DM
Estimating hpbl for 2021 at NFB
Estimating hpbl for 2021 at NM
Estimating hpbl for 2021 at NOB
Estimating hpbl for 2021 at NS
Estimating hpbl for 2021 at RM
Estimating hpbl for 2022 at DM
Estimating hpbl for 2022 at NFB
Estimating hpbl for 2022 at NM
Estimating hpbl for 2022 at NOB
Estimating hpbl for 2022 at NS
Estimating hpbl for 2022 at RM
Estimating hpbl for 2023 at DM
Estimating hpbl for 2023 at NFB
Estimating hpbl for 2023 at NM
Estimating hpbl for 2023 at NOB
Estimating hpbl for 2023 at NS
Estimating hpbl for 2023 at RM
95.2293050289154


In [5]:
import TzFuncs
import importlib
import pandas as pd
importlib.reload(TzFuncs)

# Convert FROM UTC
input = pd.date_range(start='2023-03-11 23:00',end='2023-03-12 08:00',freq='1H')
tzf = TzFuncs.Tzfuncs(Time_Zone='Etc/GMT-8')
tzf.convert(input,from_UTC=True)
print('UTC input:')
print(input)
print('Standard Time (non-TZ aware):')
print(tzf.Standard_Time)
print('Local Time (TZ aware):')
print(tzf.UTC_Time)
print('UTC Time (TZ aware):')
print(tzf.Local_Time)

# Convert FROM America/Vancouver (DST) to standard time & UTC time
input = pd.date_range(start='2023-03-11 23:00',end='2023-03-12 08:00',freq='1H')
tzf = TzFuncs.Tzfuncs(Time_Zone='America/Vancouver',DST=True)
tzf.convert(input,to_UTC=True)
print('America/Vancouver input:')
print(input)
print('Standard Time (non-TZ aware):')
print(tzf.Standard_Time)
print('Local Time (TZ & DST aware):')
print(tzf.UTC_Time)
print('UTC Time (TZ aware):')
print(tzf.Local_Time)

UTC input:
DatetimeIndex(['2023-03-11 23:00:00', '2023-03-12 00:00:00',
               '2023-03-12 01:00:00', '2023-03-12 02:00:00',
               '2023-03-12 03:00:00', '2023-03-12 04:00:00',
               '2023-03-12 05:00:00', '2023-03-12 06:00:00',
               '2023-03-12 07:00:00', '2023-03-12 08:00:00'],
              dtype='datetime64[ns]', freq='H')
Standard Time (non-TZ aware):
DatetimeIndex(['2023-03-12 07:00:00', '2023-03-12 08:00:00',
               '2023-03-12 09:00:00', '2023-03-12 10:00:00',
               '2023-03-12 11:00:00', '2023-03-12 12:00:00',
               '2023-03-12 13:00:00', '2023-03-12 14:00:00',
               '2023-03-12 15:00:00', '2023-03-12 16:00:00'],
              dtype='datetime64[ns]', freq=None)
Local Time (TZ aware):
DatetimeIndex(['2023-03-11 23:00:00+00:00', '2023-03-12 00:00:00+00:00',
               '2023-03-12 01:00:00+00:00', '2023-03-12 02:00:00+00:00',
               '2023-03-12 03:00:00+00:00', '2023-03-12 04:00:00+00:00',
        

# Make a spatial config

In [3]:
import os
import importlib
import MakeSpatialConfigFile
importlib.reload(MakeSpatialConfigFile)

for site,start_year in zip(['BB','BB2','DSM','RBM','HOGG','YOUNG'],
                            [2015,2019,2021,2022,2021,2020]):
    root = f'Y:/{site}/raw/2023/01/'
    out = f'Y:/{site}/footprint/'
    for file in os.listdir(root):
        if file.endswith('.ghg'):
            print(f'Creating site config for {site} from {root+file}')
            MakeSpatialConfigFile.from_GHG(site,root+file,start_year,out)
            break


for site,start_year in zip(['OHM'],
                           [2023]):
    root = f'Y:/{site}/raw/2023/06/'
    out = f'Y:/{site}/footprint/'
    for file in os.listdir(root):
        if file.endswith('.ghg'):
            print(f'Creating site config for {site} from {root+file}')
            MakeSpatialConfigFile.from_GHG(site,root+file,start_year,out)
            break


Creating site config for BB from Y:/BB/raw/2023/01/2023-01-01T083000_LI-7200.ghg
Creating site config for BB2 from Y:/BB2/raw/2023/01/2023-01-01T013000_AIU-1696.ghg
Creating site config for DSM from Y:/DSM/raw/2023/01/2023-01-01T053000_AIU-2264.ghg
Creating site config for RBM from Y:/RBM/raw/2023/01/2023-01-01T030000_AIU-1292.ghg
Creating site config for HOGG from Y:/HOGG/raw/2023/01/2023-01-01T063000_smart3-00494.ghg
Creating site config for YOUNG from Y:/YOUNG/raw/2023/01/2023-01-01T043000_smart3-00495.ghg
Creating site config for OHM from Y:/OHM/raw/2023/06/2023-06-06T200000_smart3-00820.ghg


In [73]:
import os
import sys
import yaml
# To find database_default from inside (the root) of a project folder
def get_matlab_default(fn="Matlab/biomet_database_default.m"):
    # Read .m file as "text"
    with open(fn,encoding='utf8') as db:
        config = db.read()
        config = config.split('if ispc')[1].split('elseif ismac')
        # Identify system, translate Matlab to Python, and evaluate
        if sys.platform == 'win32':
            exec(config[0].replace('%','#').replace(' ','').replace('\\','/'))
        elif sys.platform == 'darwin':
            exec(config[1].replace('%','#').replace(' ','').replace('\\','/'))
    return(x)

def get_config(fn='_config.yml'):
    with open(fn) as f:
        config = yaml.safe_load(f)
    return(config['Database']['root'])

os.chdir('C:/Users/User')

config_fn = '_config.yml'
matlab_fn = 'Matlab/biomet_database_default.m'

# 1 Search for _config.yml in root of Project Folder
if os.path.isfile(config_fn):
    db_root = get_config(config_fn)

# 2 Search for matalab default in root of project folder
elif os.path.isfile(matlab_fn):
    db_root = get_matlab_default(matlab_fn)

# 3 Search environment variables for UBC_PC_Setup
# Repeat 1 & 2, prompt for input as last resort
else:
    pth = [v for v in os.environ.values() if 'UBC_PC_Setup' in v]
    if len(pth)>0:
        if os.path.isfile(pth[0]+config_fn):
            db_root = get_config(pth[0]+config_fn)
        elif os.path.isfile(pth[0]+matlab_fn):
            db_root = get_matlab_default(pth[0]+matlab_fn)
    else:
        db_root = input('No default database path found, input path to database:')



C:/Database/


<module 'db_root' from 'c:\\Biomet.net\\Python\\db_root.py'>

In [76]:
# os.path.dirname(os.path.abspath(__file__))
script_path = os.path.abspath(__file__)

NameError: name '__file__' is not defined